<a href="https://colab.research.google.com/github/shuodeng521-sys/ST-554-Project1-Shuo-Anna-Jillian/blob/main/Task3/shuo_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
!pip install ucimlrepo

In [3]:
import pandas as pd
import ucimlrepo as uci
import numpy as np

In [4]:
air_quality = uci.fetch_ucirepo(id=360)

In [5]:
air = air_quality.data.features

In [6]:
air.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [7]:
air=air.rename(columns={"CO(GT)": "CO_GT", "C6H6(GT)": "C6H6_GT"})

In [8]:
cols=["C6H6_GT","CO_GT","T","RH","AH"]
air_temp=air.replace(-200,pd.NA)
air_sub=air_temp.dropna(subset=cols)
air_temp.shape, air_sub.shape


((9357, 15), (7344, 15))

In [9]:
# Select the variables you need to reduce compute time
cols = ["Date", "C6H6_GT", "CO_GT", "T", "RH", "AH"]
air_sub = air_sub[cols]
# Convert Date to datetime (important!)
air_sub["Date"] = pd.to_datetime(air_sub["Date"])
# Compute daily averages
air_daily = air_sub.groupby("Date").mean().reset_index()
#add new variable"Day"
air_daily["Day"] = range(1, len(air_daily) + 1)
air_daily.head()
air_daily.tail()

,Date,C6H6_GT,CO_GT,T,RH,AH,Day
342,2005-03-31,5.220833,1.3875,17.55,50.083333,0.951917,343
343,2005-04-01,3.526087,1.108696,16.026087,35.404348,0.631135,344
344,2005-04-02,2.529167,0.854167,15.483333,32.225,0.546167,345
345,2005-04-03,4.316667,1.141667,18.383333,33.695833,0.617583,346
346,2005-04-04,8.985714,2.078571,17.328571,41.842857,0.72125,347


In [10]:
import sklearn
reg1=sklearn.linear_model.LinearRegression()
reg1.fit(air_daily['CO_GT'].values.reshape(-1,1), air_daily['C6H6_GT'])
print(reg1.intercept_,reg1.coef_)

0.644770948344199 [4.56536116]


In [11]:
reg2=sklearn.linear_model.LinearRegression()
reg2.fit(air_daily[['CO_GT', 'T', 'RH', 'AH']],air_daily['C6H6_GT'])
print("Intercept:", reg2.intercept_)
print("Coefficients:", reg2.coef_)

Intercept: -1.8377694729981808
Coefficients: [ 4.77080433  0.11973259 -0.01620259  0.68866811]


In [15]:
from sklearn.metrics import mean_squared_error
mse_list_slr=[]
for i in range(250,len(air_daily)):
    train=air_daily.iloc[:i]
    test=air_daily.iloc[i:i+1]
    x_train=train["CO_GT"].values.reshape(-1,1)
    y_train=train["C6H6_GT"]
    x_test=test["CO_GT"].values.reshape(-1,1)
    y_test=test["C6H6_GT"]
    reg3=sklearn.linear_model.LinearRegression()
    reg3.fit(x_train,y_train)
    pred1=reg3.predict(x_test)
    mse_list_slr.append(mean_squared_error(y_test.values,pred1))
slr_mse=sum(mse_list_slr)
print('SLR total MSE:', slr_mse)

SLR total MSE: 718.0810286685651


In [18]:
mse_list_mlr=[]
for i in range(250,len(air_daily)):
    train=air_daily.iloc[:i]
    test=air_daily.iloc[i:i+1]
    x_train=train[['CO_GT', 'T', 'RH', 'AH']]
    y_train=train["C6H6_GT"]
    x_test=test[['CO_GT', 'T', 'RH', 'AH']]
    y_test=test["C6H6_GT"]
    reg4=sklearn.linear_model.LinearRegression()
    reg4.fit(x_train,y_train)
    pred2=reg4.predict(x_test)
    mse_list_mlr.append(mean_squared_error(y_test.values,pred2))
mlr_mse=sum(mse_list_mlr)
print('MLR total MSE:',mlr_mse)

MLR total MSE: 494.3422217646459


In [26]:
def one_step_mse(x,y,day):
  x_train=x.iloc[:day+1]
  y_train=y.iloc[:day+1]
  x_test=x.iloc[day+1:day+2]
  y_test=y.iloc[day+1:day+2]
  reg5=sklearn.linear_model.LinearRegression()
  reg5.fit(x_train,y_train)
  pred3=reg5.predict(x_test)
  return mean_squared_error(y_test.values,pred3)


In [29]:

  def rolling_cv(x,y,start_day=250):
      total_mse=0
      for day in range(start_day,len(x)-1):
        total_mse += one_step_mse(x,y,day)
      return total_mse

In [30]:
x_slr = air_daily[['CO_GT']]
y = air_daily['C6H6_GT']
slr_mse=rolling_cv(x_slr,y)
print("SLR total MSE",slr_mse)

SLR total MSE 707.4114126224285


In [31]:
x_mlr=air_daily[['CO_GT', 'T', 'RH', 'AH']]
y=air_daily["C6H6_GT"]
mlr_mse=rolling_cv(x_mlr,y)
print("MLR total MSE",mlr_mse)

MLR total MSE 492.71006855868444
